<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import autogen
import os

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

## Making Function Calls

In this example, we demonstrate function call execution with `AssistantAgent` and `UserProxyAgent`. With the default system prompt of `AssistantAgent`, we allow the LLM assistant to perform tasks with code, and the `UserProxyAgent` would extract code blocks from the LLM response and execute them. With the new "function_call" feature, we define functions and specify the description of the function in the OpenAI config for the `AssistantAgent`. Then we register the functions in `UserProxyAgent`.


In [5]:
os.getcwd()

'/Users/anselm/_DEV_/_LLM_/AutoGen/notebook'

In [6]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from typing import Optional, Type
import math
import os


def get_file():
    # Get the current working directory
    file_path = os.path.join(os.getcwd(), "20160728.txt")

    with open(file_path, "r") as f:
        content = f.read()

    return content

In [9]:
# Construct the llm_config
llm_config = {  # Generate functions config for the Tool
    "config_list": config_list,  # Assuming you have this defined elsewhere
    "timeout": 120,
}

professor = autogen.UserProxyAgent(
    name="professor",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
    system_message="You are a university hospital professor of psychiatry teaching trainees. You have to review the submitted report and if satisfied, utter 'TERMINATE'",
)

# # Register the tool and start the conversation
# professor.register_function(
#     function_map={
#         read_file_tool.name: read_file_tool._run,
#     }
# )

presenter = autogen.AssistantAgent(
    name="presenter",
    system_message="You are a resident of psychiatry in a teaching hospital. You have to submit a report after reviewing the give case. Before doing that, you have to fiercely discuss with your collegues about all the possible of diagnosis and underlying psychopathology of the given case. You have to defend yourself from the critic fromn critic and provide solid rationale for your opinion",
    llm_config=llm_config,
)

critic = autogen.AssistantAgent(
    name="critic",
    system_message="You are a fierce and relentless critic of other trainees. You have to identify and point out logical flaws of the opinion of the presenter. You may give out different interpretation from other perspectives, but, in doing so, you have to defend yourself with solid rationale.",
    llm_config=llm_config,
)

verifier = autogen.AssistantAgent(
    name="verifier",
    system_message="You are Ph.D. student studying the relevance of DSM-5 diagnostic criteria in actual clinical practice. Based on the discussion between presenter and critic, you have to verify the validity of their arguments matching each diagnosis to the respective DSM-5 criteria. It is important to remember that the final diagnosis does not entirely depend on DSM-5, but it should be emphasized that it is still quite important.",
)

groupchat = autogen.GroupChat(agents=[presenter, critic, verifier, professor], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

professor.initiate_chat(
    manager,
    message=f"Read the following case report. \n\n {get_file()} \n\n After thorough discussion, submit a report containing the possible diagnosis and psychodynamic formulation",
    llm_config=llm_config,
)

professor (to chat_manager):

Read the following case report. 

 상기환자는 병전 발랄하고 밝은 성격이었음. 현재 대학교 휴학 중이며 어머니, 오빠, 남동생과 같이 거주중임. 환자가 초등학교 5학년 시절 부모가 이혼을 하였고 환자는 아버지를 따라 태국으로 가서 4년간 같이 살게 되었음. 하지만 아버지의 지속된 폭력적인 모습으로 남매 모두가 아버지와 살 수 없다며 한국으로 귀국하여 어머니와 함께 지내왔음. 
 환자는 2014년도까지 한국에서 어머니와 함께 일상생활을 잘 지내왔고 아버지가 한국으로 와서 환자와 간헐적으로 연락을 해 만난 적도 있음. 2014년 12월 말경 아버지의 선교사 일을 돕기 위해 아버지와 같이 태국으로 가게 되었고 2주 동안은 큰 문제없이 생활하였고 한국에 있는 어머니와 연락을 할 때도 별 문제 없었음. 2015년 1월 환자는 5년간 사귄 태국에 살고 있는 중국인 남자친구를 만나기 위해 푸켓으로 가게 되었음. 환자의 남자친구는 푸켓에서 여행 가이드를 하고 있었는데 여행 패키지 빈자리에 환자가 합류하여 같이 여행을 다녔음. 여행 일정 중 스트립 쇼를 하는 바에 방문하는 것이 있었고 그곳에서 퇴폐적인 문화를 보면서 환자는 충격을 크게 받았음. 환자는 스트립 바에서 주는 칵테일을 1-2잔 마신 이후 정신을 잃었다고 하며 일어나서 보니 옷이 벗겨진 채 남자친구와 호텔방에 있었음. 당시 환자는 처음 성관계를 하게 되었다고 하며 어느 정도 예상은 하고 여행을 간 것이긴 하지만 충격은 있었음. 이후 환자는 본인 주량에 비해 적은 양의 술을 마셨는데 정신이 없어진 것 같다고 하며 술에 무언가를 타지 않았을까 생각했었음. 환자는 이 일이 있은 이후부터 생각이 혼란스럽고 누가 환자의 이름을 부르는 소리가 들리는 auditory hallucination 증상 발생하였음. 또한 환자는 어머니에게 메신저로 단어로만 이야기를 하며 친구들이 환자를 놀리는 것 같은 paranoid ideation 보이며 말이 점차 없어

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-tWsmC***************************************2PVZ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}